<h1>Teste da resolução do zombies utilizando Decision Tree e Random Forest</h1>

Iremos utilizar primeiro o um decision tree classifier para verificar a taxa de acerto e, após isso, iremos utilizar um random forest classifier. Também será realizado ao decorrer do notebook algumas análises dos dados.

In [1]:
import pandas as pd
ds = pd.read_csv("zombie-health-cases500.csv")
ds_original = ds.copy()

In [2]:
ds.head(10)

,paralysis,yellow_tongue,trembiling_finger,member_loss,chest_pain,severe_anger,diagnostic
0,f,f,f,f,t,f,viral_infection
1,f,t,t,f,f,f,zulombriga
2,f,f,f,t,t,f,viral_infection
3,f,f,t,f,f,f,zulombriga
4,f,t,t,f,f,f,zulombriga
5,f,t,f,f,f,f,zulombriga
6,f,t,t,f,f,f,zulombriga
7,f,f,f,t,t,f,viral_infection
8,f,f,f,f,t,f,viral_infection
9,t,t,f,f,f,f,bacterial_infection


<h3>Transformando nossos dados em inteiros</h3>

In [3]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
for i in range(len(ds.columns) - 1):
    ds.iloc[:, i] = enc.fit_transform(ds.iloc[:,i])
ds.head(10)

,paralysis,yellow_tongue,trembiling_finger,member_loss,chest_pain,severe_anger,diagnostic
0,0,0,0,0,1,0,viral_infection
1,0,1,1,0,0,0,zulombriga
2,0,0,0,1,1,0,viral_infection
3,0,0,1,0,0,0,zulombriga
4,0,1,1,0,0,0,zulombriga
5,0,1,0,0,0,0,zulombriga
6,0,1,1,0,0,0,zulombriga
7,0,0,0,1,1,0,viral_infection
8,0,0,0,0,1,0,viral_infection
9,1,1,0,0,0,0,bacterial_infection


<h3> Realizando one hot encoding na coluna diagnostic </h3>

Iremos transformar nossa coluna diagnostic em inteiros para podermos prevê-los futuramente, além disso iremos realizar o one hot encoding na mesma coluna para tratar as variáveis nominais.

Agora nosso dataframe está pronto para serem realizadas análises em cima dele

In [4]:
one_hot = pd.get_dummies(ds['diagnostic'])
ds['diagnostic'] = enc.fit_transform(ds['diagnostic'])
ds = ds.join(one_hot)
ds.head(20)

,paralysis,yellow_tongue,trembiling_finger,member_loss,chest_pain,severe_anger,diagnostic,bacterial_infection,fight,viral_infection,zulombriga
0,0,0,0,0,1,0,2,0,0,1,0
1,0,1,1,0,0,0,3,0,0,0,1
2,0,0,0,1,1,0,2,0,0,1,0
3,0,0,1,0,0,0,3,0,0,0,1
4,0,1,1,0,0,0,3,0,0,0,1
5,0,1,0,0,0,0,3,0,0,0,1
6,0,1,1,0,0,0,3,0,0,0,1
7,0,0,0,1,1,0,2,0,0,1,0
8,0,0,0,0,1,0,2,0,0,1,0
9,1,1,0,0,0,0,0,1,0,0,0


<h4> Agora podemos analisar a tabela relacional dos nossos dados </h4>


In [24]:
ds.corr()

,paralysis,yellow_tongue,trembiling_finger,member_loss,chest_pain,severe_anger,diagnostic,bacterial_infection,fight,viral_infection,zulombriga
paralysis,1.000000,0.398386,-0.226690,-0.021866,-0.200445,-0.234867,-0.454148,0.580886,-0.249787,-0.104984,-0.240300
yellow_tongue,0.398386,1.000000,0.168906,-0.136208,-0.597356,-0.405866,-0.330635,0.713886,-0.431648,-0.457392,0.176698
trembiling_finger,-0.226690,0.168906,1.000000,-0.322825,-0.405353,-0.275412,0.713466,-0.305610,-0.292907,-0.310377,0.943360
member_loss,-0.021866,-0.136208,-0.322825,1.000000,0.145282,0.290364,-0.245755,0.007113,0.295551,0.020804,-0.330892
chest_pain,-0.200445,-0.597356,-0.405353,0.145282,1.000000,0.134683,0.148508,-0.466024,0.147556,0.728431,-0.429691
severe_anger,-0.234867,-0.405866,-0.275412,0.290364,0.134683,1.000000,-0.224556,-0.316634,0.940270,-0.321573,-0.291948
diagnostic,-0.454148,-0.330635,0.713466,-0.245755,0.148508,-0.224556,1.000000,-0.783892,-0.238820,0.289992,0.756303
bacterial_infection,0.580886,0.713886,-0.305610,0.007113,-0.466024,-0.316634,-0.783892,1.000000,-0.336748,-0.356832,-0.323958
fight,-0.249787,-0.431648,-0.292907,0.295551,0.147556,0.940270,-0.238820,-0.336748,1.000000,-0.342001,-0.310494
viral_infection,-0.104984,-0.457392,-0.310377,0.020804,0.728431,-0.321573,0.289992,-0.356832,-0.342001,1.000000,-0.329012


<h2> Iremos criar um identificador de relações específicas </h2>

Para termos as menos quantidades de perguntas para identificar a doença do zumbi, a ideia é que identifiquemos quais sintomas são mais relacionados a quais doenças. Assim, podemos ir verificando, com mais certeza, qual a doença do paciente a partir de poucas perguntas.

A ideia do algoritmo é justamente guardar o sintoma com mais correlação e a doença, e, automaticamente, montar uma cadeia de if's para resolver o problema.

<b>Obs: </b>o programa ainda está em teste então não tem nada automatizado

In [49]:
def achar_maior(corr):
    max = 0
    for i in range(4):
        for j in range(6):
           if corr.iloc[i,j] > max:
                max = corr.iloc[i,j]
                sintoma = j
                doenca = i
    corr.iloc[doenca, sintoma] = 0
    return max, sintoma, doenca

In [60]:
sintomas = ['paralysis', 'yellow_tongue', 'trembiling_finger', 'member_loss', 'chest_pain', 'severe_anger']
doencas = ['bacterial infection', 'fight', 'viral infection', 'zulombriga']

In [61]:
corr = ds.corr()
corr = corr.iloc[7: , :6]
nos = []
for z in range(6):
    nos.append(achar_maior(corr))

In [71]:
new_sintoma = [0, 1, 0, 0, 0, 0]
for i in range(len(nos)):
    print('Doutor: Você sente', str(sintomas[nos[i][1]]) + str('?')) 
    if new_sintoma[nos[i][1]] == 1:
        print('Zumbi: Yes')
        print('Diagnóstico:', doencas[nos[i][2]])
        break
    else:
        print('Zumbi: No')

Doutor: Você sente trembiling_finger?
Zumbi: No
Doutor: Você sente severe_anger?
Zumbi: No
Doutor: Você sente chest_pain?
Zumbi: No
Doutor: Você sente yellow_tongue?
Zumbi: Yes
Diagnóstico: bacterial infection


<h2> Depois de tratar os dados, podemos finalmente começar as predições </h2>

Irei utilizar aqui uma abordagem que eu tirei da minha cabeça para ver se funciona. O problema é que estamos lidando com valores nominais, então não conseguimos realizar uma boa predição com apenas um modelo de árvore de decisão. A minha ideia é montar um número de modelos igual ao número de doenças possíveis e comparar o resultado entre eles de uma instância. Ou seja, iremos fazer 4 árvores de decisão e ver qual me devolve uma doença com mais certeza.

É importante ressaltar que teremos de usar o DecisionTreeRegressor ao invés do DecisionTreeClassifier, pois precisamos de uma probabilidade e não de um classificador, isto é, para que a ideia funcione precisamos usar a função sigma no lugar da função step.

<b>Obs:</b> O código está muito mal implementado por enquanto, mas só quero testar se a ideia funciona.

In [6]:
sintomas = ['paralysis', 'yellow_tongue', 'trembiling_finger', 'member_loss', 'chest_pain', 'severe_anger']

In [7]:
x = ds[sintomas]
y = ds.bacterial_infection

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=0)

from sklearn.ensemble import RandomForestRegressor
model_bacteria = RandomForestRegressor()
model_bacteria.fit(x_train, y_train)

C:\Users\Okita\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [9]:
model_bacteria.score(x_test, y_test)

0.9633289451525144

In [10]:
y = ds.fight
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=0)
model_fight = RandomForestRegressor()
model_fight.fit(x_train, y_train)
model_fight.score(x_test, y_test)

0.8398412611280598

In [11]:
y = ds.viral_infection
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=0)
model_viral = RandomForestRegressor()
model_viral.fit(x_train, y_train)
model_viral.score(x_test, y_test)

0.8436422708077134

In [12]:
y = ds.zulombriga
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=0)
model_zulombriga = RandomForestRegressor()
model_zulombriga.fit(x_train, y_train)
model_zulombriga.score(x_test, y_test)

0.9167109028096087

In [100]:
Xnew = [[0, 1, 1, 0, 0, 0]]

ynew = model_bacteria.predict(Xnew)
print("Modelo Bacteria")
print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))

ynew = model_fight.predict(Xnew)
print("Modelo Luta")
print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))

ynew = model_viral.predict(Xnew)
print("Modelo Viral")
print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))

ynew = model_zulombriga.predict(Xnew)
print("Modelo Zulombriga")
print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))

Modelo Bacteria
X=[0, 1, 1, 0, 0, 0], Predicted=0.0
Modelo Luta
X=[0, 1, 1, 0, 0, 0], Predicted=0.0
Modelo Viral
X=[0, 1, 1, 0, 0, 0], Predicted=0.0
Modelo Zulombriga
X=[0, 1, 1, 0, 0, 0], Predicted=1.0


<h2> Segunda opinião </h2>

Irei utilizar o random forest como uma segunda opinião à encontrada pelo doutor. O ponto é que o médico pode fazer perguntas, porém não consegue ter tanta precisão quanto nosso random forest - que por sua vez não seria tão eficiente nas perguntas. Então a ideia é que utilizemos o random forest para verificar o diagnóstico do médico e, caso haja, sugerir uma possível segunda doença que pode existir.

In [82]:
def diagnostico_preciso(model, paciente, first, second):
    ynew = model.predict(paciente)
    if ynew > first:
        second = first
        first = ynew
        return first, second, 1
    elif ynew > second:
        second = ynew
        return first, second, 2
    return first, second, 3

In [103]:
paciente = [[0, 1, 0, 0, 0, 0]]
doenca = ['', '']
first = 0
second = 0

aux = diagnostico_preciso(model_bacteria, paciente, first, second)
if aux[2] == 1:
    doenca[1], doenca[0] = doenca[0], 'bacteria'
    first, second = aux[0], aux[1]
elif aux[2] == 2:
    doenca[1] = 'bacteria'

aux = diagnostico_preciso(model_fight, paciente, first, second)
if aux[2] == 1:
    doenca[1], doenca[0] = doenca[0], 'luta'
    first, second = aux[0], aux[1]
elif aux[2] == 2:
    doenca[1] = 'luta'

aux = diagnostico_preciso(model_viral, paciente, first, second)
if aux[2] == 1:
    doenca[1], doenca[0] = doenca[0], 'virus'
    first, second = aux[0], aux[1]
elif aux[2] == 2:
    doenca[1] = 'virus'

aux = diagnostico_preciso(model_zulombriga, paciente, first, second)
if aux[2] == 1:
    doenca[1], doenca[0] = doenca[0], 'zulombriga'
    first, second = aux[0], aux[1]
elif aux[2] == 2:
    doenca[1] = 'zulombriga'

for i in range(len(nos)):
    print('Doutor: Você sente', str(sintomas[nos[i][1]]) + str('?')) 
    if paciente[0][nos[i][1]] == 1:
        print('Zumbi: Yes')
        print('Diagnóstico:', doencas[nos[i][2]])
        break
    else:
        print('Zumbi: No')
        
print('\n -------------------------------- \n')
print('SEGUNDA OPINIÃO')
print('Diagnostico mais provável:', doenca[0])
if doenca[1] != '':
    print('Diagnostico também possível:', doenca[1])

Doutor: Você sente trembiling_finger?
Zumbi: No
Doutor: Você sente severe_anger?
Zumbi: No
Doutor: Você sente chest_pain?
Zumbi: No
Doutor: Você sente yellow_tongue?
Zumbi: Yes
Diagnóstico: bacterial infection

 -------------------------------- 

SEGUNDA OPINIÃO
Diagnostico mais provável: bacteria
Diagnostico também possível: zulombriga
